In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

model = load_model("stray_dogs_detector.model")
CATEGORIES = ["not_stray_dogs", "stray_dogs"]

# Load object detection model (MobileNet SSD)
prototxt_path = r"C:\AI Course tamil\14.Deep Learning\Dogs_monitoring\dog_detector\MobileNetSSD_deploy.prototxt"
weights_path = r"C:\AI Course tamil\14.Deep Learning\Dogs_monitoring\dog_detector\MobileNetSSD_deploy.caffemodel"
net = cv2.dnn.readNetFromCaffe(prototxt_path, weights_path)

# Dog class ID in MobileNet SSD
DOG_CLASS_ID = 12

# --- SELECT VIDEO SOURCE ---
# Set to 0 for webcam or provide video file path
VIDEO_SOURCE = r"C:\AI Course tamil\14.Deep Learning\Dogs_monitoring\for video detection.mp4"  # Replace with your video path or use 0 for webcam

cap = cv2.VideoCapture(VIDEO_SOURCE)
if not cap.isOpened():
    print("[ERROR] Could not open video source.")
    exit()

print("[INFO] Processing video... Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("[INFO] End of video or can't read frame.")
        break

    h, w = frame.shape[:2]

    # Object detection
    blob = cv2.dnn.blobFromImage(frame, 0.007843, (300, 300), 127.5)
    net.setInput(blob)
    detections = net.forward()

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        class_id = int(detections[0, 0, i, 1])

        if confidence > 0.5 and class_id == DOG_CLASS_ID:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            startX = max(0, startX)
            startY = max(0, startY)
            endX = min(w, endX)
            endY = min(h, endY)

            dog_roi = frame[startY:endY, startX:endX]
            if dog_roi.size == 0:
                continue

            try:
                dog_img = cv2.resize(dog_roi, (224, 224))
                dog_img = img_to_array(dog_img)
                dog_img = preprocess_input(dog_img)
                dog_img = np.expand_dims(dog_img, axis=0)

                pred = model.predict(dog_img)[0]
                idx = np.argmax(pred)
                label = CATEGORIES[idx]
                confidence_str = f"{label}: {pred[idx]*100:.2f}%"
                color = (0, 255, 0) if label == "not_stray_dogs" else (0, 0, 255)

                cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
                cv2.putText(frame, confidence_str, (startX, startY - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
            except Exception as e:
                print(f"[WARNING] Error processing ROI: {e}")

    cv2.imshow("Stray Dog Detector", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
